<a href="https://colab.research.google.com/github/shravan1994/shravan1994/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Mounting drive and copying the model_training.ipynb file from drive 
<p>This file contains few methods to build a features, which we want to reuse in this file</p>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/MyDrive/Colab Notebooks/Self case study/Talking Data User Demographics Prediction/submissions/model_training.ipynb' .

!jupyter nbconvert 'model_training.ipynb' --to python

# after running this file, all it's functions will be availble in current file to run.
# import model_training.ipynb
import model_training

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[NbConvertApp] Converting notebook model_training.ipynb to python
[NbConvertApp] Writing 34630 bytes to model_training.py
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 50 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
Is events data available?    What we are predicting?                                               Test log loss
---------------------------  ------------------------------------------------------------------  ---------------
Yes                          Gender                                                                     0.491647
Yes                          User group (used predicted gender from above model as new feature)     

In [3]:
# drive path to load models from
drive_path = '/content/drive/MyDrive/Colab Notebooks/Self case study/Talking Data User Demographics Prediction/models'

## Downloading data from kaggle

In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "."

!kaggle competitions download -c talkingdata-mobile-user-demographics

!unzip '/content/talkingdata-mobile-user-demographics.zip'
!unzip '/content/app_events.csv.zip' -d data
!unzip '/content/app_labels.csv.zip' -d data
!unzip '/content/events.csv.zip' -d data
!unzip '/content/gender_age_test.csv.zip' -d data
!unzip '/content/gender_age_train.csv.zip' -d data
!unzip '/content/label_categories.csv.zip' -d data
!unzip '/content/phone_brand_device_model.csv.zip' -d data

 95% 268M/283M [00:01<00:00, 163MB/s]
100% 283M/283M [00:01<00:00, 162MB/s]
Archive:  /content/talkingdata-mobile-user-demographics.zip
  inflating: app_events.csv.zip      
  inflating: app_labels.csv.zip      
  inflating: events.csv.zip          
  inflating: gender_age_test.csv.zip  
  inflating: gender_age_train.csv.zip  
  inflating: label_categories.csv.zip  
  inflating: phone_brand_device_model.csv.zip  
  inflating: sample_submission.csv.zip  
Archive:  /content/app_events.csv.zip
  inflating: data/app_events.csv     
Archive:  /content/app_labels.csv.zip
  inflating: data/app_labels.csv     
Archive:  /content/events.csv.zip
  inflating: data/events.csv         
Archive:  /content/gender_age_test.csv.zip
  inflating: data/gender_age_test.csv  
Archive:  /content/gender_age_train.csv.zip
  inflating: data/gender_age_train.csv  
Archive:  /content/label_categories.csv.zip
  inflating: data/label_categories.csv  
Archive:  /content/phone_brand_device_model.csv.zip
  inflating: 

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer

from geopy.distance import geodesic, great_circle
from scipy.sparse import hstack, save_npz
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

In [6]:
def select_k_best_(features, label, n=1000, training=False):
  from sklearn.feature_selection import SelectKBest, chi2
  import pickle
  filename = f'{drive_path}/select_k_best_{label}.pkl'

  kbest = pickle.load(open(filename, 'rb'))
  best_features = kbest.transform(features)
  return best_features

## This method encodes the target labels and returns the same

In [7]:
def get_labels_(y, label='group', inverse=False):
  from sklearn.preprocessing import LabelEncoder
  import pickle

  filename = f'{drive_path}/label_encoder_{label}.pkl'
  enc = pickle.load(open(filename, 'rb'))
  if inverse:
    y = enc.inverse_transform(y)
  else:
    y =  enc.transform(y)
  return y

## This method returns the all features, based on weather it has events or not.

In [8]:
def get_features(devices_data, raw_data_dict, label, has_events=False):
  all_events_df = raw_data_dict.get('all_events')
  all_app_events_df = raw_data_dict.get('all_app_events')
  app_labels_df = raw_data_dict.get('app_labels')
  label_categories_df = raw_data_dict.get('label_categories')
  phone_brand_df = raw_data_dict.get('phone_brand')
  
  phone_brand_resp_enc = model_training.get_phone_brand_features_response_enc(
      devices_data, phone_brand_df, None, label=f'{label}_no_events', training=False)
  device_model_resp_enc = model_training.get_device_model_features_response_enc(
      devices_data, phone_brand_df, None, label=f'{label}_no_events', training=False)
  bow_phone_brand = model_training.get_phone_brand_features_bow(devices_data, phone_brand_df, training=False)
  bow_device_model = model_training.get_device_model_features_bow(devices_data, phone_brand_df, training=False)
  
  if not has_events:
    X = hstack([bow_phone_brand, bow_device_model, phone_brand_resp_enc, device_model_resp_enc])
    
    print('data shape before removing no var data: ', X.shape)
    X = model_training.remove_features_with_no_variance(X, training=False, label=f'{label}_no_events')
    print('data shape', X.shape)
    return X
  else:
    categories_bow = model_training.get_bag_of_categories(
      devices_data, all_events_df, all_app_events_df, 
      app_labels_df, label_categories_df, training=False
    )
    
    apps_bow = model_training.get_bag_of_apps(devices_data, app_labels_df, all_events_df, all_app_events_df, label_categories_df, training=False)
    
    total_events = model_training.get_total_events(devices_data, all_events_df, training=False)
    total_apps = model_training.total_apps_installed(devices_data, all_events_df, all_app_events_df, training=False)
    distance_travelled = model_training.distance_traveled_per_day(devices_data, all_events_df, training=False)
    median_lat_long = model_training.get_median_lat_long(devices_data, all_events_df)
    hourly_events = model_training.get_hourly_events(devices_data, all_events_df)
    
    apps_bow_best = select_k_best_(apps_bow, label=label, n=2000, training=False)

    X = hstack([categories_bow, apps_bow_best, 
            bow_phone_brand, bow_device_model, 
            total_apps, distance_travelled, median_lat_long, hourly_events])

    print('data shape before removing no variance data: ', X.shape)
    X = model_training.remove_features_with_no_variance(X, training=False, label=f'{label}_with_events')
    print('data shape', X.shape)
  
    return X

## We use the existing trained model to predict gender on devices with events data

In [9]:
def predict_gender_with_events(devices_with_events, raw_data_dict):
  print('Predicting gender using events data')
  import pickle
  file_name = f"{drive_path}/best_model_gender_with_events.pkl"
  cali_cfl = pickle.load(open(file_name, "rb"))

  X_data = get_features(devices_with_events, raw_data_dict, label='gender', has_events=True)
  
  y_gen_pred = cali_cfl.predict(X_data)
  y_gen_pred_proba = cali_cfl.predict_proba(X_data)
  
  return y_gen_pred, y_gen_pred_proba


In [25]:
def predict_user_group_with_events(devices_with_events, predicted_gender, raw_data_dict):
  print('Predicting user group using events data')
  import pickle
  file_name = f"{drive_path}/best_model_group_with_events.pkl"
  X_data = get_features(devices_with_events, raw_data_dict, label='group', has_events=True)
  X_data = hstack([X_data, predicted_gender])

  cali_cfl = pickle.load(open(file_name, "rb"))
  predict_y_proba = cali_cfl.predict_proba(X_data)
  predict_y = cali_cfl.predict(X_data)
  
  print('classes: ', cali_cfl.classes_)

  return predict_y, predict_y_proba

In [11]:
def predict_gender_no_events(devices_no_events, raw_data_dict):
  print('Predicting gender on devices with no events')
  import pickle
  file_name = f"{drive_path}/best_model_gender_with_no_events.pkl"
  
  X_data = get_features(devices_no_events, raw_data_dict, label='gender', has_events=False)
    
  cali_cfl = pickle.load(open(file_name, "rb"))
  predict_y_test_proba = cali_cfl.predict_proba(X_data)
  predict_y_test = cali_cfl.predict(X_data)

  return predict_y_test, predict_y_test_proba

In [12]:
def predict_user_groups_no_events(devices_no_events, predicted_gender, raw_data_dict):
  print('Predicting user group on devices with no events')
  import pickle
  file_name = f"{drive_path}/best_model_group_with_no_events.pkl"
    
  X_data = get_features(devices_no_events, raw_data_dict, label='group', has_events=False)
  X_data = hstack([X_data, predicted_gender])
  
  cali_cfl = pickle.load(open(file_name, "rb"))
  predict_y_proba = cali_cfl.predict_proba(X_data)
  predict_y = cali_cfl.predict(X_data)

  return predict_y, predict_y_proba

In [ ]:
# def train_user_group_events_nb(X_data, y_tr_events, training=False):
#   import pickle
#   file_name = f"{drive_path}/best_model_group_predictor_with_events_nb.pkl"

#   if training:
#     parameters = {'alpha': [0.00001, 0.0005, 0.0001, 0.005, 0.001, 0.05, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100, 1000, 10000]}
  
#     nb_clf = MultinomialNB()
#     grid_search = GridSearchCV(nb_clf, parameters, cv=5, scoring='neg_log_loss', verbose=10, n_jobs=-1)
#     grid_search.fit(X_data, y_tr_events)

#     print('best params: ', grid_search.best_params_)

#     nb_model = grid_search.best_estimator_

#     sig_clf = CalibratedClassifierCV(nb_model, method="sigmoid")
#     sig_clf.fit(X_data, y_tr_events)

#     y_pred = sig_clf.predict(X_data)
#     y_pred_proba = sig_clf.predict_proba(X_data)

#     # saving the model to disk for later usage
#     pickle.dump(sig_clf, open(file_name, "wb"))
#     return y_pred, y_pred_proba
#   else:
#     sig_clf = pickle.load(open(file_name, "rb"))
#     y_pred = sig_clf.predict(X_data)
#     y_pred_proba = sig_clf.predict_proba(X_data)
#     return y_pred, y_pred_proba

## final_func_1(), this method returns predicted labels and predicted probabilities 

In [29]:
def final_fun_1(devices, raw_data_dict, with_eval=False):
  all_events_df = raw_data_dict.get('all_events')
  is_events = model_training.has_events(devices, all_events_df)

  final_y_pred_proba = np.zeros((devices.shape[0], 12))
  final_y_pred = np.zeros(devices.shape[0])

  devices_with_events = devices[is_events == 1]

  # predicting gender on devices with events
  y_gen_pred, y_gen_pred_proba = predict_gender_with_events(devices_with_events[['device_id']], raw_data_dict)
  devices_with_events['gen_pred'] = y_gen_pred
  devices_with_events[['gen_proba_1', 'gen_proba_2']] = y_gen_pred_proba

  if with_eval:
    y_gender_events_true = get_labels_(devices_with_events['gender'].values, label='gender')
    print('Logloss for gender prediction with events data: ', log_loss(y_gender_events_true, y_gen_pred_proba))
  
  # predicting group on devices with events
  y_with_events_pred, y_with_events_pred_proba = predict_user_group_with_events(devices_with_events[['device_id']], y_gen_pred_proba, raw_data_dict)
  devices_with_events['pred_group'] = y_with_events_pred
  devices_with_events[[f'pred_group_proba_{column}' for column in range(0, 12)]] = y_with_events_pred_proba

  if with_eval:
    y_group_events_true = get_labels_(devices_with_events['group'].values, label='group')
    print('Logloss for user group prediction with events data: ', log_loss(y_group_events_true, y_with_events_pred_proba))

  # storing predicted values in final_y_pred_proba and final_y_pred, at correct indexes, 
  # so that predicted values are in same order that of passed devices 
  with_events_prob_indx = np.expand_dims(np.where(is_events == 1)[0], axis=1)
  np.put_along_axis(final_y_pred_proba, with_events_prob_indx, y_with_events_pred_proba, axis=0)
  
  with_events_indx = np.where(is_events == 1)[0]
  np.put_along_axis(final_y_pred, with_events_indx, y_with_events_pred, axis=0)

  devices_no_events = devices[is_events == 0]

  # predicting gender on devices without events
  y_gen_pred_no_events, y_gen_pred_no_events_proba = predict_gender_no_events(devices_no_events[['device_id']], raw_data_dict)
  devices_no_events['gen_pred'] = y_gen_pred_no_events
  devices_no_events[['gen_proba_1', 'gen_proba_2']] = y_gen_pred_no_events_proba

  if with_eval:
    y_gender_noevents_true = get_labels_(devices_no_events['gender'].values, label='gender')
    print('Logloss for gender prediction without events data: ', log_loss(y_gender_noevents_true, y_gen_pred_no_events_proba))

  # predicting gender on devices without events
  y_pred_no_events, y_pred_no_events_proba = predict_user_groups_no_events(devices_no_events[['device_id']], y_gen_pred_no_events_proba, raw_data_dict)
  devices_no_events['pred_group'] = y_pred_no_events
  devices_no_events[[f'pred_group_proba_{column}' for column in range(0, 12)]] = y_pred_no_events_proba

  if with_eval:
    y_group_noevents_true = get_labels_(devices_no_events['group'].values, label='group')
    print('Logloss for user group prediction without events data: ', log_loss(y_group_noevents_true, y_pred_no_events_proba))

  no_events_indx_proba = np.expand_dims(np.where(is_events == 0)[0], axis=1)
  np.put_along_axis(final_y_pred_proba, no_events_indx_proba, y_pred_no_events_proba, axis=0)

  no_events_indx = np.where(is_events == 0)[0]
  np.put_along_axis(final_y_pred, no_events_indx, y_pred_no_events, axis=0)

  devices = devices.merge(devices_with_events, on='device_id', how='left').merge(devices_no_events, on='device_id', how='left')
  print(devices.head())
  return devices['pred_group'].values, devices['pred_group_proba'].values

## final_func_2(), this method returns log loss

In [14]:
def final_fun_2(devices, raw_data_dict):
  all_events_df = raw_data_dict.get('all_events')
  y_true = get_labels_(devices['group'].values, label='group')

  y_pred, y_pred_proba = final_fun_1(devices, raw_data_dict, with_eval=True)
  print(len(y_true))
  print(len(y_pred_proba))
  return log_loss(y_true, y_pred_proba)

In [19]:
devices = pd.read_csv('data/gender_age_train.csv')

raw_data_dict = {
    'all_events': pd.read_csv('data/events.csv'),
    'all_app_events': pd.read_csv('data/app_events.csv'),
    'app_labels':  pd.read_csv('data/app_labels.csv'),
    'label_categories': pd.read_csv('data/label_categories.csv'),
    'phone_brand':  pd.read_csv('data/phone_brand_device_model.csv')
}

final_fun_1(devices, raw_data_dict, with_eval=True)

Predicting gender using events data
distance travelled  (23309, 9)
data shape before removing no variance data:  (23309, 4591)
data shape (23309, 2843)
Logloss for gender prediction with events data:  0.45691175448394333
Predicting user group using events data
distance travelled  (23309, 9)
data shape before removing no variance data:  (23309, 4591)
data shape (23309, 2842)
Logloss for user group prediction with events data:  1.8396423837234026
Predicting gender on devices with no events
data shape before removing no var data:  (51336, 1731)
data shape (51336, 1043)
Logloss for gender prediction without events data:  0.6285619334348501
Predicting user group on devices with no events
data shape before removing no var data:  (51336, 1751)
data shape (51336, 1068)
Logloss for user group prediction without events data:  2.2915588224168753


(array([ 8.,  7., 10., ..., 10.,  7.,  7.]),
 array([[0.04547512, 0.05536524, 0.03763794, ..., 0.09790811, 0.12324675,
         0.08764882],
        [0.04824545, 0.07218755, 0.03882167, ..., 0.10708565, 0.13826961,
         0.09667036],
        [0.14991827, 0.0527382 , 0.0436365 , ..., 0.10408701, 0.16573645,
         0.10779792],
        ...,
        [0.06418519, 0.05895201, 0.0390226 , ..., 0.10050034, 0.12459151,
         0.11046935],
        [0.04569494, 0.04571339, 0.0331571 , ..., 0.10314492, 0.19358975,
         0.05878446],
        [0.03836587, 0.04205361, 0.03883689, ..., 0.14967962, 0.17404905,
         0.06732349]]))

In [20]:
devices = pd.read_csv('data/gender_age_train.csv')

raw_data_dict = {
    'all_events': pd.read_csv('data/events.csv'),
    'all_app_events': pd.read_csv('data/app_events.csv'),
    'app_labels':  pd.read_csv('data/app_labels.csv'),
    'label_categories': pd.read_csv('data/label_categories.csv'),
    'phone_brand':  pd.read_csv('data/phone_brand_device_model.csv')
}

final_fun_2(devices, raw_data_dict, with_eval=True)

Predicting gender using events data
distance travelled  (23309, 9)
data shape before removing no variance data:  (23309, 4591)
data shape (23309, 2843)
Logloss for gender prediction with events data:  0.45691175448394333
Predicting user group using events data
distance travelled  (23309, 9)
data shape before removing no variance data:  (23309, 4591)
data shape (23309, 2842)
Logloss for user group prediction with events data:  1.8396423837234026
Predicting gender on devices with no events
data shape before removing no var data:  (51336, 1731)
data shape (51336, 1043)
Logloss for gender prediction without events data:  0.6285619334348501
Predicting user group on devices with no events
data shape before removing no var data:  (51336, 1751)
data shape (51336, 1068)
Logloss for user group prediction without events data:  2.2915588224168753
74645
74645


2.1504412623725835

## Observation

*   final_func_2 returns the result log loss on training data is : 2.1519659987248203



In [30]:
devices = pd.read_csv('data/gender_age_test.csv')

raw_data_dict = {
    'all_events': pd.read_csv('data/events.csv'),
    'all_app_events': pd.read_csv('data/app_events.csv'),
    'app_labels':  pd.read_csv('data/app_labels.csv'),
    'label_categories': pd.read_csv('data/label_categories.csv'),
    'phone_brand':  pd.read_csv('data/phone_brand_device_model.csv')
}

y_pred, y_pred_proba = final_fun_1(devices, raw_data_dict)
result = pd.DataFrame(y_pred_proba, index=devices.device_id, columns=get_labels_([0,1,2,3,4,5,6,7,8,9,10,11], inverse=True))
result.head()
result.to_csv('submission_file.csv')

Predicting gender using events data
distance travelled  (35194, 9)
data shape before removing no variance data:  (35194, 4591)
data shape (35194, 2843)
Predicting user group using events data
distance travelled  (35194, 9)
data shape before removing no variance data:  (35194, 4591)
data shape (35194, 2842)
classes:  [ 0  1  2  3  4  5  6  7  8  9 10 11]
Predicting gender on devices with no events
data shape before removing no var data:  (76877, 1731)
data shape (76877, 1043)
Predicting user group on devices with no events
data shape before removing no var data:  (76877, 1751)
data shape (76877, 1068)
             device_id  gen_pred_x  gen_proba_1_x  gen_proba_2_x  \
0  1002079943728939269         1.0       0.114819       0.885181   
1 -1547860181818787117         1.0       0.255693       0.744307   
2  7374582448058474277         0.0       0.677073       0.322927   
3 -6220210354783429585         1.0       0.138893       0.861107   
4 -5893464122623104785         NaN            NaN   

KeyError: ignored